In [7]:
import datetime
import pytz

import matplotlib.pyplot as plt
import mplcyberpunk

from pathlib import Path
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.wcs as fitswcs
import astropy.units as u
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, get_moon, Angle
from astropy.time import Time
from astropy.table import Table
import astroplan
from specutils.spectra.spectrum1d import Spectrum1D
from astropy.visualization import quantity_support
from specutils.manipulation import FluxConservingResampler

from sklearn.decomposition import PCA

_ = quantity_support()
plt.style.use("cyberpunk")
%matplotlib widget

In [2]:
df = pd.read_csv("sky_spectra.csv")
moonfree = df[(df['moon_alt'] < -10) & (df['disperser'] == '270_gpm')]
moonfree

,file,ut,alt,az,moon_alt,moon_az,moon_ill,disperser
1,./2004.0413/skysub_a2034.bg_1_comb.fits,2004-04-13T08:25:25,81.18,75.47,-17.42,107.26,37,270_gpm
3,./2004.0413/skysub_F2.20.bg_1_comb.fits,2004-04-13T05:53:13,56.71,278.43,-47.97,91.12,38,270_gpm
4,./2004.0414/skysub_F2.bg_1_comb.fits,2004-04-14T05:11:49,65.04,274.96,-65.06,67.04,28,270_gpm
5,./2004.0414/skysub_F2.bg_2_comb.fits,2004-04-14T07:00:59,41.07,284.86,-43.32,87.36,27,270_gpm
6,./2004.0415/skysub_F2.bg_3_comb.fits,2004-04-15T05:48:01,55.37,278.13,-64.16,53.43,19,270_gpm
...,...,...,...,...,...,...,...,...
4125,./2020.1013/skysub_m31_pne_20_9_comb.fits,2020-10-13T08:16:47,66.13,300.54,-16.84,60.23,17,270_gpm
4128,./2020.1024/skysub_jiang_c2_1_comb.fits,2020-10-24T12:04:06,65.47,80.09,-59.19,282.18,60,270_gpm
4131,./2020.1024/skysub_Rank13_1_comb.fits,2020-10-24T08:14:37,50.94,204.84,-13.27,252.44,58,270_gpm
4132,./2020.1025/skysub_jiang_c2_1_comb.fits,2020-10-25T11:29:55,59.13,78.40,-41.03,273.43,69,270_gpm


In [3]:
new_disp_grid = np.arange(3850, 6800, 2) * u.AA
fluxcon = FluxConservingResampler()

In [4]:
fluxes = []
for i, r in moonfree.iterrows():
    sp = Spectrum1D.read(r['file'])
    trim_sp = fluxcon(sp, new_disp_grid)
    fluxes.append(trim_sp.flux)

In [6]:
fluxes = np.array(fluxes)
fits.writeto("sky_fluxes.fts", fluxes, overwrite=True)

In [5]:
fluxes = fits.open('sky_fluxes.fts')[0].data

In [8]:
plt.figure(figsize=[15,8])
plt.yscale('log')
plt.step(new_disp_grid, np.mean(fluxes, axis=0))
plt.step(new_disp_grid, np.std(fluxes, axis=0))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, np.mean(fluxes, axis=0)/np.std(fluxes, axis=0))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, 2.5 * np.log10(fluxes[-1, :] / fluxes[2200, :]))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
pca = PCA(n_components=16).fit(fluxes)

In [12]:
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
components = pca.transform(fluxes)
filtered = pca.inverse_transform(components)

In [14]:
components.shape

(2309, 16)

In [15]:
filtered.shape

(2309, 1475)

In [16]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, fluxes[-4, :])
plt.step(new_disp_grid, fluxes[-4, :] - filtered[-4, :])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
features = Table.read("features.csv")
plt.figure(figsize=[25,16])
#plt.yscale('log')
#for i in range(5):
    #plt.step(new_disp_grid, pca_sm.components_[i], label=f"Component {i}")
i = 0
plt.step(new_disp_grid, pca.components_[i], label=f"Component {i}")
txt_y = 1.02 * np.max(pca.components_[i])
for f in features:
    if f['low wave'] != f['high wave']:
        plt.axvspan(f['low wave'], f['high wave'], color='r', alpha=0.1)
        txt_x = 0.5*(f['low wave'] + f['high wave'])
        plt.text(txt_x, 1.05*txt_y, f['name'], horizontalalignment='center', verticalalignment='top')
    else:
        x = f['low wave']
        plt.text(x, txt_y, f"{f['name']} {x}", rotation='vertical', fontsize='medium', horizontalalignment='right', verticalalignment='top')
        plt.axvline(x, color='black', alpha=0.2)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
components[-1, :]

array([ 1.32715163,  1.21202569, -2.11438958, -1.10036221,  0.04728834,
       -0.02333088,  0.43063577,  0.38794274,  0.14074741, -0.50295542,
       -0.10110243,  0.05056153,  0.01697325,  0.04346665, -0.05780361,
       -0.02984308])

In [19]:
components[0, :]

array([-2.09881474e-01, -2.29452258e+00,  4.50188326e-01, -1.04057383e+00,
       -1.24920801e-01, -1.72296578e-02, -1.47683673e-01, -1.47547598e-01,
       -2.82452537e-02, -5.38782421e-02, -4.15297818e-03, -2.63035772e-04,
        1.87318502e-02, -5.91421061e-03,  6.69949939e-03, -4.42054275e-03])

In [20]:
np.arange(16).reshape(4, 4).flatten()[3]

3

In [21]:
fig, axs = plt.subplots(4, 4, figsize=[25,16], sharex=True)
for i, ax in enumerate(axs.flat):
    ax.step(new_disp_grid, pca.components_[i])
    ax.set_title(f"Component {i}")
    ax.set_ylabel("counts/s")
    ax.set_xlabel("Wavelength (A)")
    if not ax.is_last_row():
        #ax.set_xticklabels([])
        ax.set_xlabel("")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
components[:, 1].std()

2.6502438723773056

In [23]:
plt.figure()
plt.scatter(moonfree['alt'], components[:, 7])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
np.argmax(components[:, 0])

1294

In [196]:
moonfree.iloc[998]

file         ./2010.1001/skysub_hecto_spec_2010_1_comb.fits
ut                                      2010-10-01T03:32:18
alt                                                    52.7
az                                                    23.63
moon_alt                                             -27.28
moon_az                                                34.6
moon_ill                                                 50
disperser                                           270_gpm
Name: 1646, dtype: object

In [25]:
resid = fluxes - filtered

In [26]:
rms = resid.std()
plt.figure(figsize=[6,6])
plt.imshow(resid, vmin=-3*rms, vmax=3*rms)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
resid.std()

In [67]:
new_disp_grid[1132]

<Quantity 6114. Angstrom>

In [27]:
new_disp_grid[919]

<Quantity 5688. Angstrom>

In [28]:
new_disp_grid[254]

<Quantity 4358. Angstrom>

In [29]:
fits.writeto("resid.fits", resid)